# Using Jurassic-2 Grande Instruct on SageMaker through Model Packages

This sample notebook shows you how to deploy **Jurassic-2 Grande Instruct** using Amazon SageMaker.

## Pre-requisites:
1. Before running this notebook, please make sure you got this notebook from the model catalog on SageMaker AWS Management Console.
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**.
1. This notebook is intended to work with **boto3 v1.25.4** or higher.

## Contents:
1. [Select model package](#1.-Select-model-package)
1. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   1. [Interact with the model](#B.-Interact-with-the-model)
   1. [Give it an instruction](#C.-Give-it-an-instruction)
   1. [Create your own product description!](#D.-Create-your-own-product-description!)
1. [Clean-up](#3.-Clean-up)
   1. [Delete the endpoint](#A.-Delete-the-endpoint)
   1. [Delete the model](#B.-Delete-the-model)
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Select model package
Confirm that you received this notebook from the model catalog in SageMaker AWS Management Console.

In [20]:
model_package_map = {
    "us-east-1": "arn:aws:sagemaker:us-east-1:865070037744:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "us-east-2": "arn:aws:sagemaker:us-east-2:057799348421:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "us-west-1": "arn:aws:sagemaker:us-west-1:382657785993:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "us-west-2": "arn:aws:sagemaker:us-west-2:594846645681:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "ca-central-1": "arn:aws:sagemaker:ca-central-1:470592106596:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "eu-central-1": "arn:aws:sagemaker:eu-central-1:446921602837:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "eu-west-1": "arn:aws:sagemaker:eu-west-1:985815980388:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "eu-west-2": "arn:aws:sagemaker:eu-west-2:856760150666:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "eu-west-3": "arn:aws:sagemaker:eu-west-3:843114510376:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "eu-north-1": "arn:aws:sagemaker:eu-north-1:136758871317:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "ap-southeast-1": "arn:aws:sagemaker:ap-southeast-1:192199979996:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "ap-southeast-2": "arn:aws:sagemaker:ap-southeast-2:666831318237:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "ap-northeast-2": "arn:aws:sagemaker:ap-northeast-2:745090734665:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "ap-northeast-1": "arn:aws:sagemaker:ap-northeast-1:977537786026:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "ap-south-1": "arn:aws:sagemaker:ap-south-1:077584701553:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "sa-east-1": "arn:aws:sagemaker:sa-east-1:270155090741:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378"
}

In [21]:
import json
from sagemaker import ModelPackage
from sagemaker import get_execution_role
import sagemaker as sage
import boto3

### Check the version of boto3 - must be v1.25.4 or higher
If you see a lower version number, pick another kernel to run the notebook, with Python 3.8 or above

In [22]:
boto3.__version__

'1.26.74'

### Install ai21 python SDK

In [23]:
! pip install -U "ai21[SM]"
import ai21

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [24]:
region = boto3.Session().region_name
if region not in model_package_map.keys():
    raise ("UNSUPPORTED REGION")

model_package_arn = model_package_map[region]

In [25]:
role = get_execution_role()
sagemaker_session = sage.Session()

runtime_sm_client = boto3.client("runtime.sagemaker")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html).

In [26]:
endpoint_name = "j2-grande-instruct"

content_type = "application/json"

real_time_inference_instance_type = (
    "ml.p4d.24xlarge"
)

### A. Create an endpoint

In [27]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=endpoint_name, 
                         model_data_download_timeout=3600,
                         container_startup_health_check_timeout=600,
                        )

-----------------------------!

Once endpoint has been created, you would be able to perform real-time inference.

### B. Interact with the model

You can think of Jurassic-2 Grande Instruct as a smart auto-completion algorithm: give it some text as input and it will generate relevant text to naturally complete your input.

These two helpful concepts are worth being familiar with:
- **Prompt** - the input you provide to the model.
- **Completion** - the output text the model generates.

Enter a simple prompt: "To be, or", and let the model complete it

In [28]:
response = ai21.Completion.execute(sm_endpoint=endpoint_name,
                                   prompt="To be, or",
                                   maxTokens=4,
                                   temperature=0,
                                   numResults=1)

print(response['completions'][0]['data']['text'])

 not to be: that is the question


As you can see, the model identifies the beginning of a famous quote, and completes it correctly.

### C. Give it an instruction

This model was specifically trained to follow natural language instructions. It is the most natural way to interact with large language models: simply tell the model what you want it to do, and it will follow.

For this notebook, we will apply the model to creating a product description for an eCommerce site. Simply tell the model what you want it to do with a simple instruction.

In [29]:
instruction = """Write an engaging product description for clothing eCommerce site.
Product: Humor Men's Graphic T-Shirt.
Description:

"""

response = ai21.Completion.execute(sm_endpoint=endpoint_name,
                                   prompt=instruction,
                                   maxTokens=100,
                                   temperature=0,
                                   numResults=1)

print(response['completions'][0]['data']['text'])

This funny t-shirt is perfect for anyone who enjoys a good laugh. The graphic on the front features an image of a cat with a mustache and glasses, and the text reads "I mustache you a question." The shirt is made of 100% cotton and is soft and comfortable. It's sure to become a favorite in your wardrobe.


#### Adjust the parameters
A useful parameter is the temperature. **You can increase creativity by tweaking the temperature.** With temperature 0, the model will always choose the most probable completion, so it will always be the same. Increasing the temperature will provide varying completions, where the completion may be different with every generation. We recommend starting with a value of 0.7.

In [30]:
instruction = """Write an engaging product description for clothing eCommerce site.
Product: Humor Men's Graphic T-Shirt.
Description:

"""

response = ai21.Completion.execute(sm_endpoint=endpoint_name,
                                   prompt=instruction,
                                   maxTokens=100,
                                   temperature=0.7,
                                   numResults=3)

# Note that multiple completions - this is due to numResults > 1
for comp in response['completions']:
    print(comp['data']['text'].strip())
    print("=============")

This funny t-shirt is perfect for any guy with a sense of humor. It's made of high-quality cotton and it's comfortable to wear. The graphics are printed using the latest technology, so they won't fade or crack. The shirt comes in a variety of colors, so you can choose the one that best fits your personality. It's a great gift for any occasion, and it's sure to bring a smile to your face.
This funny t-shirt is perfect for any guy who has a sarcastic sense of humor. The graphic design features a skull wearing a crown and the words "I'm the King." It's sure to get laughs and make a bold statement.

The shirt is made from high-quality cotton and is comfortable to wear. It's available in a range of sizes, so you can find the perfect fit for you.
This funny T-shirt is perfect for any occasion. The graphic print reads "I Just Can't Keep Calm, I'm A Mom" and is sure to bring a smile to your face. The 100% cotton construction is soft and comfortable, and the relaxed fit allows for easy movement

#### Be specific in your prompt
You can see that the model generated several versions. There may be times when you are looking for a stricter format - such as a single paragraph. Just ask the model to stick to it:

In [31]:
instruction = """Write an engaging product description for clothing eCommerce site. Make it a single paragraph.
Product: Humor Men's Graphic T-Shirt.
Description:

"""

response = ai21.Completion.execute(sm_endpoint=endpoint_name,
                                   prompt=instruction,
                                   maxTokens=100,
                                   temperature=0.5,
                                   numResults=3)

for comp in response['completions']:
    print(comp['data']['text'].strip())
    print("=============")

This funny men's graphic t-shirt is perfect for any occasion. The shirt features a humorous saying on the front and is made from a soft and comfortable material. It's sure to become a favorite in your wardrobe.
This funny t-shirt is perfect for anyone who enjoys a good laugh. The graphic on the front is of a grumpy old man, and the text says "I Don't Like Mondays." It's made of soft, comfortable cotton and comes in a variety of colors.
This funny t-shirt is perfect for anyone who loves a good laugh. It's made of 100% cotton, so it's comfortable to wear, and the graphics are printed on the front. The shirt is available in a variety of colors, so you can choose the one that best fits your style.


#### Personalize your prompt
The model can generate nice descriptions based on only an instruction, but they may not be aligned with your actual product. If you have special features, ask the model to include them:

In [32]:
instruction = """Write an engaging product description for clothing eCommerce site. Make sure to include the following features in the description.
Product: Humor Men's Graphic T-Shirt with a print of Einstein's quote: "artificial intelligence is no match for natural stupidity”
Features:
- Soft cotton
- Short sleeve
Description:

"""

response = ai21.Completion.execute(sm_endpoint=endpoint_name,
                                   prompt=instruction,
                                   maxTokens=100,
                                   temperature=0.7,
                                   numResults=3)

for comp in response['completions']:
    print(comp['data']['text'].strip())
    print("=============")

Einstein's famous quote about artificial intelligence being no match for natural stupidity is printed on this men's t-shirt. It's a fun way to show your disdain for AI and your support for human intelligence.
Einstein's famous quote about AI is the perfect way to show off a sense of humor. This men's t-shirt is made of soft cotton and is comfortable to wear. The print is bold and eye-catching, and is sure to be a conversation starter.
Are you a smart person? Do you enjoy making witty jokes and sarcastic comments? If so, this Humor Men's Graphic T-Shirt is perfect for you! The print on the front reads "Artificial intelligence is no match for natural stupidity", and it's sure to get a chuckle from those around you.

The shirt is made of soft cotton and comes in a variety of colors. It's comfortable to wear and easy to clean. Whether you're wearing it to work or out with friends, this shirt is sure to make a statement.

So, what are you waiting for? Get your Humor Men's Graphic T-Shirt to

#### More format restrictions?
If you have several examples of your optimal output, you can feed the model with a few-shot prompt that includes several examples. Prompts like this are very useful when you want to enforce a certain format that is non-trivial to pick up from just one example. These prompts work best with base models; try to use them with [Jurassic-2 Grande](https://github.com/AI21Labs/SageMaker/blob/main/J2_Grande_example_model_use.ipynb) instead.

### D. Create your own product description!

Want to build your own product description? Here's your chance!

In [33]:
# Functions to build the prompt
def list_features(features):
    return "\n".join(
        f"- {feat}" for feat in features
    )

def build_prompt(product, features):
    example = "Write product descriptions for fashion eCommerce site. Make sure to include the following features in the description.\n"
    example += f"Product: {product}\n"
    example += "Features:\n"
    example += list_features(features)
    example += "\nDescription:\n"
    
    return example

In [34]:
product = input("Product: (for example, \'Women's Boho Beach Dress\')")
features = input("Features, separated by commas with no spaces: (for example, \'midi dress,swing hem,special for summer\')")
your_prompt = build_prompt(product, features.split(","))

Product: (for example, 'Women's Boho Beach Dress') Women's Boho Beach Dress
Features, separated by commas with no spaces: (for example, 'midi dress,swing hem,special for summer') midi dress,swing hem,special for summer


In [35]:
response = ai21.Completion.execute(sm_endpoint=endpoint_name,
                                   prompt=your_prompt,
                                   maxTokens=100,
                                   temperature=0.7,
                                   numResults=3)

for comp in response['completions']:
    print(comp['data']['text'].strip())
    print("=============")

This boho beach dress is perfect for summer days. It's made of lightweight fabric and has a swing hem. The midi length makes it comfortable to wear, and the print is eye-catching.
The Boho Beach Dress is perfect for hot summer days. This midi dress has a swing hem and is made from lightweight fabric. The dress has a bohemian style that is perfect for beach vacations.
This boho beach dress is perfect for summer. The midi dress has a beautiful swing hem and the floral pattern is eye-catching. The fabric is lightweight and breathable, making it comfortable to wear even in hot weather.


### Interested in learning more?
Take a look at our [blog post](https://www.ai21.com/blog/building-cv-profile-generator-using-ai21-studio) to understand the process of building a good prompt.

## 3. Clean-up

### A. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [36]:
model.sagemaker_session.delete_endpoint(endpoint_name)
model.sagemaker_session.delete_endpoint_config(endpoint_name)

### B. Delete the model

In [37]:
model.delete_model()